## Particle Tracking Experiment ##


__Can we look at regimes before and after September 8__

[x] - Add Bathymetry - Tidal Bar?  
    - Using 15 arcsecond ETOPO1  
[x] - Add Surface Vectors  
[x] - 24, 48 hours  
[] - Save Data format  


In [1]:
from opendrift.readers import reader_netCDF_CF_generic
from opendrift.readers import reader_global_landmask
from opendrift.models.oceandrift import OceanDrift
import datetime as dt
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import geopandas as gpd
import pandas as pd
import shapely,tqdm,glob,cmocean, os

__Configure data and landmask__

[Seed Shape File](https://www.keene.edu/campus/maps/tool/?coordinates=-122.4794504%2C%2037.8086990%0A-122.4821978%2C%2037.8236160%0A-122.5319934%2C%2037.8127676%0A-122.5663352%2C%2037.8298532%0A-122.5567195%2C%2037.7563299%0A-122.5244382%2C%2037.7742427%0A-122.5096712%2C%2037.7897093%0A-122.4797939%2C%2037.8089702%0A-122.4794504%2C%2037.8086990)

[Bolinas](https://www.keene.edu/campus/maps/tool/?coordinates=-122.5655149%2C%2037.8299887%0A-122.5461075%2C%2037.8398857%0A-122.5754603%2C%2037.8640125%0A-122.6578989%2C%2037.9127841%0A-122.6977442%2C%2037.9114297%0A-122.7032402%2C%2037.8951758%0A-122.7585427%2C%2037.8564229%0A-122.6001919%2C%2037.7943215%0A-122.5661859%2C%2037.8303955%0A-122.5655149%2C%2037.8299887)

[Drake's Bay](https://www.keene.edu/campus/maps/tool/?coordinates=-122.7584005%2C%2037.8553386%0A-122.7013804%2C%2037.9041159%0A-122.7851931%2C%2037.9902102%0A-122.8374042%2C%2038.0426836%0A-122.9260257%2C%2038.0480911%0A-123.0057164%2C%2038.0026557%0A-123.0146473%2C%2037.9956216%0A-123.0558666%2C%2037.9349915%0A-122.7570265%2C%2037.8526278%0A-122.7584005%2C%2037.8553386)

[SF Penninsula](https://www.keene.edu/campus/maps/tool/?coordinates=-122.5570993%2C%2037.7567371%0A-122.5112428%2C%2037.7770920%0A-122.4719122%2C%2037.6691468%0A-122.4932088%2C%2037.5886629%0A-122.5206884%2C%2037.5924715%0A-122.5694646%2C%2037.5908393%0A-122.5577857%2C%2037.7549727)

In [2]:
def load_roi_shapefiles():
    sf_penninsula = gpd.read_file("./data/bay_zones/sf_peninsula.json",driver='GeoJSON',features='sf_peninsula')
    gg_mouth = gpd.read_file("./data/bay_zones/sf-bay-seed.json",driver='GeoJSON',features='sf_bay-seed')
    bolinas = gpd.read_file("./data/bay_zones/bolinas.json",driver='GeoJSON',features='bolinas')
    drakes = gpd.read_file("./data/bay_zones/drakes_region.json",driver='GeoJSON',features='drakes_region')
    gdf = pd.concat([sf_penninsula,gg_mouth,bolinas,drakes])
    gdf['name'] = ['sf_peninsula','gg_mouth','bolinas','drakes']
    return gdf

def particle_tracking(date):
    o = OceanDrift(loglevel=50)
    reader_landmask = reader_global_landmask.Reader()
    url = './data/surface_currents/hfr-sfbay-2024-april.nc'
    o.add_reader(reader_netCDF_CF_generic.Reader(url))
    o.add_reader(reader_landmask)
    o.set_config('general:coastline_approximation_precision', .001)  # approx 100m
    o.set_config('general:coastline_action', 'stranding') 
    o.set_config('general:time_step_minutes',15)
    o.set_config('general:time_step_output_minutes',30)
    o.set_config('drift:advection_scheme', 'runge-kutta')
    o.set_config('drift:stokes_drift', False)
    o.set_config('drift:current_uncertainty_uniform', .1)
    o.set_config('seed:ocean_only', False)
    o.seed_from_shapefile("./data/seed_shapefile/sf-bay-seed-small-polygon.shp",number=50,time=date,layername=None)
    o.run(steps=48*4) # 24 hours
    return o


def particle_in_polygon(model_output,gdf,time_step):
    """ Estimate the reatlive portion of particles in different predefined regions"""
    lons = model_output.history['lon']
    lats = model_output.history['lat']
    
    llns = lons[:,time_step]
    llns = llns[llns.mask == False]
    lts = lats[:,time_step]
    lts = lts[lts.mask == False]
    
    bolinas = 0
    mouth = 0
    peninsula = 0
    drakes = 0
    total = len(llns)
    for ln,lt in zip(llns,lts):
        out = gdf.contains(shapely.geometry.Point(ln,lt))
        if out.sum() > 0:
            if gdf.loc[out,'name'].values[0] == 'bolinas':
                bolinas = bolinas + 1
            elif gdf.loc[out,'name'].values[0] == 'gg_mouth':
                mouth = mouth + 1
            elif gdf.loc[out,'name'].values[0] == 'sf_peninsula':
                peninsula = peninsula + 1
            elif gdf.loc[out,'name'].values[0] == 'drakes':
                drakes = drakes + 1
                
    return [mouth/total, bolinas/total, peninsula/total, drakes/total]


def load_bathy_data():
    """ 
    Load Bathymetry data (.tiff) from outside the SF Bay Area
    
    """
    ds = xr.open_dataset('./data/bathymetry/sf_bay_topo.tiff',engine='rasterio')
    elv = ds['band_data'].values
    xx = ds.x.values
    yy = ds.y.values
    return xx,yy,elv


def load_surface_currents(fname='./data/surface_currents/hfr-sfbay-2024-april.nc',):
    """
    Load HFR surface currents data from the SF Bay Area
    """
    ds = xr.open_dataset('./data/surface_currents/hfr-sfbay-2024-april.nc')
    ds = ds.sel(time=slice(start_date,start_date+dt.timedelta(hours=48)),lat=slice(37.5,38),lon=slice(-123,-122.2))
    ds = ds[['u','v']]
    return ds


def make_map(xx,yy,elv):
    fig, ax = plt.subplots(1,subplot_kw={'projection': ccrs.PlateCarree()})
    fig.set_size_inches(8,8)
    cmap = cmocean.cm.haline

    ax.add_feature(cfeature.LAND,zorder=-1)
    ax.add_feature(cfeature.COASTLINE,zorder=-1)


    ax.set_xlim(-123, -122.35)
    ax.set_ylim(37.5, 38.1)

    
    gl = ax.gridlines(draw_labels=True, linestyle='--',zorder=-3)
    gl.top_labels = False
    gl.right_labels = False


    levels = [0,10,20,50,100,200,500,1000]
    ax.contourf(xx,yy,-1*elv[0],zorder=-2,cmap=cmocean.cm.ice_r,levels=levels)
    cont = ax.contour(xx,yy,-1*elv[0],levels=levels[1:],colors='k',linewidths=1,lw='solid')
    ax.clabel(cont, inline=True, fontsize=10, fmt='%1.0f')

    # Create a custom colorbar
    norm = mcolors.Normalize(vmin=0, vmax=96)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    # Add colorbar to the plot
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal' ,anchor=(0.81,2.6),shrink=0.15,aspect=10)
    cbar.set_ticks([0,48,96])
    cbar.set_ticklabels(['0','24','48'],fontweight='bold')
    cbar.set_label('Hours', fontsize=10, fontweight='bold', labelpad=-40)

    return fig, ax


## Plotting
__Static Plot__

In [5]:
def generate_static_plot(o,start_date):
    lons = o.history['lon']
    lats = o.history['lat']
    cmap = cmocean.cm.haline
    xx,yy,elv = load_bathy_data()
    ds = load_surface_currents()


    fig, ax = make_map(xx,yy,elv)

    for track in range(0, lats.shape[0]):
        llns = lons[track,:]
        llns = llns[llns.mask == False]
        lts = lats[track,:]
        lts = lts[lts.mask == False]
        
        ax.scatter(llns[0],lts[0],edgecolors='k',s=20,facecolors='k',zorder=100,marker='.')
        
        if llns.shape[0] <  48:
            last_known = llns.shape[0] -1
        
        else:
            last_known = 48
            
        ax.plot(llns[:last_known-1],lts[:last_known-1],c='.5',lw=1)
        
        ax.scatter(llns[last_known-1], lts[last_known-1],
                edgecolors='k',
                s=40,
                c=cmap(last_known/96),
                zorder=100,
                marker='o')
        
        if (last_known == 48) & (llns.shape[0] ==  96):
            last_known = 96
        else:
            last_known = llns.shape[0] -1
        
        if last_known > 48:
            ax.plot(llns[:last_known-1],lts[:last_known-1],c='.5',lw=1,ls='--')
            ax.scatter(llns[last_known-1], lts[last_known-1],
                edgecolors='k',
                s=40,
                c=cmap(last_known/96),
                zorder=100,
                marker='o')
            
            
    day_str = (start_date).strftime("%Y-%m-%d")
    hour_str = (start_date).strftime("%H:%M")
    ax.text(1,1.05,day_str,fontweight='bold',fontsize=18,transform=ax.transAxes,ha='right',va='bottom')
    ax.text(1,1,hour_str,fontweight='bold',fontsize=18,transform=ax.transAxes,ha='right',va='bottom')

    plt.savefig(f"./figures/static/sf_plume_static_{day_str}_res.png",bbox_inches='tight',pad_inches=0.1)


def generate_animation_img_stack(o, start_date, add_current_vectors=False):
    
    # Remove all files in the temp_img_stack folder
    if add_current_vectors:
        output_dir = './figures/temp_img_stack/vector'
    else:
        output_dir = './figures/temp_img_stack/no_vector'
    delete_list = glob.glob(os.path.join(output_dir,"*.png"))
    os.system(f"rm {' '.join(delete_list)}")
    
    
    if add_current_vectors:
        hfr_current_vectors = load_surface_currents()    
        
    lons = o.history['lon']
    lats = o.history['lat']
    cmap = cmocean.cm.haline
    xx,yy,elv = load_bathy_data()
    gdf = load_roi_shapefiles()

    for hours in tqdm.tqdm(range(96)):
    # hours = 24
        fig, ax = make_map(xx,yy,elv)

        # Plot starting points
        llns = lons[:,0]
        llns = llns[llns.mask == False]
        lts = lats[:,0]
        lts = lts[lts.mask == False]
        ax.scatter(llns,lts,zorder=100,c='k',alpha=.5,s=20,marker='.')

        # Plot particle position at current Hour
        llns = lons[:,hours]
        llns = llns[llns.mask == False]
        lts = lats[:,hours]
        lts = lts[lts.mask == False]
        ax.scatter(llns,lts,c=cmap(hours/48),s=20)

        # Plot pervious 6 hours trajectory
        for j in range(lons.shape[0]):
            llns = lons[j,:]
            llns = llns[llns.mask == False]
            lts = lats[j,:]
            lts = lts[lts.mask == False]
            if llns.shape[0] > 1:
                if hours > 6:
                    ax.plot(llns[hours-6:hours],lts[hours-6:hours],c='.25',lw=2,zorder=-1)
                else:
                    ax.plot(llns[:hours],lts[:hours],c='.25',lw=2,zorder=-1)


        # Add Zones
        for loc in gdf['geometry']:
            ax.add_geometries([loc], crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black',linewidth=2,zorder=-2)


        # Date and Hour Text
        day_str = (start_date + dt.timedelta(minutes=30)*hours).strftime("%Y-%m-%d")
        hour_str = (start_date + dt.timedelta(minutes=30)*hours).strftime("%H:%M")
        ax.text(1,1.05,day_str,fontweight='bold',fontsize=18,transform=ax.transAxes,ha='right',va='bottom')
        ax.text(1,1,hour_str,fontweight='bold',fontsize=18,transform=ax.transAxes,ha='right',va='bottom')


        # Inset Percentage Particle Plot
        ins = ax.inset_axes([0.45,0.75,0.2,0.2])
        values = particle_in_polygon(o,gdf,hours)
        values = [round(values[i]*100) for i in range(4)]
        ins.bar([1,2,3,4],values,align='center')
        ins.set_xticks([1,2,3,4])
        ins.set_xlim(0.5,4.5)
        ins.set_ylim(0,100)
        ins.set_ylabel('% Particles', fontdict={'fontweight':'bold'}, labelpad=-7 )
        ins.set_yticks([0,50,100])
        ins.set_yticklabels(['0','50','100'],fontweight='bold')
        ins.xaxis.set_ticklabels(['M','BLS','SF ','Dra'],fontweight='bold')
        ins.patch.set_facecolor('None')
        sns.despine(ax=ins)

        if add_current_vectors:
            vectors = hfr_current_vectors.sel(time=start_date+dt.timedelta(minutes=30*hours),method='nearest')[['u','v']]
            ax.quiver(hfr_current_vectors.lon, 
                      hfr_current_vectors.lat, 
                      vectors.u, 
                      vectors.v, 
                      scale=10)

        # Save Figure
        if add_current_vectors:
            plt.savefig(f"./figures/temp_img_stack/vector/{start_date.year}_sf_{str(hours).zfill(2)}_vector_res.png",bbox_inches='tight',pad_inches=0.1)
        
        else:
            plt.savefig(f"./figures/temp_img_stack/no_vector/{start_date.year}_sf_{str(hours).zfill(2)}_res.png",bbox_inches='tight',pad_inches=0.1)
        
        plt.close()
    #Make anitimation from Image Stack
    if add_current_vectors:
        cmd = f"convert -delay 10 -loop 0 $(ls -1v ./figures/temp_img_stack/vector/*.png) ./figures/animations/{start_date.strftime('%Y%m%dT%H%M%S')}_48_hour_vector.gif"
    
    else:
        cmd = f"convert -delay 10 -loop 0 $(ls -1v ./figures/temp_img_stack/no_vector/*.png) ./figures/animations/{start_date.strftime('%Y%m%dT%H%M%S')}_48_hour.gif"
    os.system(cmd)



__Run Moodel__

In [7]:
start_date =dt.datetime(2024,4,12,12)
o = particle_tracking(start_date)

In [ ]:
generate_static_plot(o,start_date)
generate_animation_img_stack(o, start_date, add_current_vectors=True)
generate_animation_img_stack(o, start_date, add_current_vectors=False)

In [8]:
generate_animation_img_stack(o,start_date)

  0%|          | 0/96 [00:00<?, ?it/s]

100%|██████████| 96/96 [00:49<00:00,  1.95it/s]


In [ ]:
sns.set_context("talk")
lons_2d = lons.reshape(30,30,49)
lats_2d = lats.reshape(30,30,49)
# single_day = ds.sel(time=start_date,method='nearest')
fig, ax = plt.subplots(1,subplot_kw={'projection': ccrs.PlateCarree()})
fig.set_size_inches(8,9)
# single_day['speed'] = np.sqrt(single_day['u_mean']**2 + single_day['v_mean']**2) * 100

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
cmap = plt.cm.Blues_r
cmap_inner = plt.cm.Reds_r


ax.set_xlim(-122.5,-121.74)
ax.set_ylim(36.4,37)
gl = ax.gridlines(draw_labels=True, linestyle='--')
gl.top_labels = False
gl.right_labels = False

res_time,lon,lat = o.get_residence_time(5000)
cax = ax.contourf(lon[1:],lat[1:],res_time.T/60,levels=[5,10,15,20,30],zorder=-1)

cbar = plt.colorbar(cax,pad=0.05,shrink=.4,extendrect=True,orientation="horizontal",anchor=(.8,6.5),drawedges=False)
cbar.set_label("hours",labelpad=-60)
ax.set_xlim(-122.5,-121.74)
ax.set_ylim(36.4,37)

# ax.text(x=0.05,y=.95,s="2022-05-25\n48 hr Residence Time",verticalalignment='top', transform=ax.transAxes)
# plt.savefig("./figures/relaxation_residences_2022.png", dpi=300)
ax.text(x=0.05,y=.95,s=f"{start_date}\n48 hr Residence Time",verticalalignment='top', transform=ax.transAxes)
# plt.savefig("./figures/upwelling_residences_2022.png", dpi=300)



In [ ]:
!pwd

In [ ]:
total_length, distances, speeds = o.get_trajectory_lengths()

total_length.reshape(30,30)

sns.set_context("paper")
lons_2d = lons.reshape(30,30,49)
lats_2d = lats.reshape(30,30,49)
# single_day = ds.sel(time=start_date,method='nearest')
fig, ax = plt.subplots(1,subplot_kw={'projection': ccrs.PlateCarree()})
fig.set_size_inches(8,8)
# single_day['speed'] = np.sqrt(single_day['u_mean']**2 + single_day['v_mean']**2) * 100

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
cmap = plt.cm.Blues_r
cmap_inner = plt.cm.Reds_r


i = 0
llns = lons_2d[::2,:,i]
llns = llns[llns.mask == False]
lts = lats_2d[::2,:,i]
lts = lts[lts.mask == False]

cax = ax.scatter(llns,lts,zorder=100,c=total_length.reshape(30,30)[::2,:],alpha=1,s=20)
cbar = plt.colorbar(cax,pad=0.05,shrink=.5)
cbar.set_label("Trajectory Distance [m]")

ax.set_xlim(-122.5,-121.74)
ax.set_ylim(36.4,37)
ax.gridlines(draw_labels=True, linestyle='--',)

# ax.scatter(lons[:,-1], lats[:,-1])
# single_day.plot.quiver(x='lon',y="lat",u="u_mean",v="v_mean",ax=ax)
# single_day['speed'].plot(ax=ax)
# single_day.plot.quiver(x='lon',y="lat",u="u_mean",v="v_mean",ax=ax)

# ax.set_title(f"{single_day.time.values}")



In [ ]:
avg_speed = np.mean(speeds,axis=0).reshape(30,30)

sns.set_context("paper")
lons_2d = lons.reshape(30,30,49)
lats_2d = lats.reshape(30,30,49)
# single_day = ds.sel(time=start_date,method='nearest')
fig, ax = plt.subplots(1,subplot_kw={'projection': ccrs.PlateCarree()})
fig.set_size_inches(8,8)
# single_day['speed'] = np.sqrt(single_day['u_mean']**2 + single_day['v_mean']**2) * 100

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
cmap = plt.cm.Blues_r
cmap_inner = plt.cm.Reds_r


i = 0
llns = lons_2d[::2,:,i]
llns = llns[llns.mask == False]
lts = lats_2d[::2,:,i]
lts = lts[lts.mask == False]

cax = ax.scatter(llns,lts,zorder=100,c=avg_speed[::2,:],alpha=1,s=20)
cbar = plt.colorbar(cax,pad=0.05,shrink=.5)
cbar.set_label("Average Particle Speed [m]")

ax.set_xlim(-122.5,-121.74)
ax.set_ylim(36.4,37)
ax.gridlines(draw_labels=True, linestyle='--',)

In [ ]:
res_time,lon,lat = o.get_residence_time(5000)
xx,yy = np.meshgrid(lon[:-1],lat[:-1])
sns.set_context("paper")

# single_day = ds.sel(time=start_date,method='nearest')
fig, ax = plt.subplots(1,subplot_kw={'projection': ccrs.PlateCarree()})
fig.set_size_inches(8,8)
# single_day['speed'] = np.sqrt(single_day['u_mean']**2 + single_day['v_mean']**2) * 100

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
cmap = plt.cm.Blues_r
cmap_inner = plt.cm.Reds_r


res_time,lon,lat = o.get_residence_time(5000)
cax = ax.contour(lon[1:],lat[1:],res_time.T/60,levels=[5,10,15,20,30])
# cbar = plt.colorbar(cax,pad=0.05,shrink=.5)
# cbar.set_label("Residence Time [hrs]")

ax.set_xlim(-122.5,-121.74)
ax.set_ylim(36.3,37)
ax.gridlines(draw_labels=True, linestyle='--',)




In [ ]:
1830.0/60

In [ ]:
fig, ax = plt.subplots()
ax.contourf(lon[:-1],lat[:-1],res_time.T)

In [ ]:
single_day = ds.sel(time=slice('20211012','20211030'))

# for i in range(24):
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
fig.set_size_inches(10,10)
make_map(single_day.isel(time=i), ax)
ax.set_xlim(-122.75,-121.75)

In [ ]:
x = ds.longitude.values
y = ds.latitude.values
# Fudge to make evenly spaced
X = np.linspace(min(x),max(x),len(x))
Y = np.linspace(min(y),max(y),len(y))

xx,yy = np.meshgrid(X,Y)
norm = mcolors.Normalize(vmin=0,vmax=1)

__Get Wire Walker Data__

In [ ]:
fname = "/Users/patrick/OneDrive/UCSC/IFCB/power-buoy/wire-walker-profiler/processed/ww_power_buoy_casts_hourly.csv"
df = pd.read_csv(fname,index_col=["cast_hour",'depth'])
df.head()

__Get data into grid for countour plots__

In [ ]:
temp_gridded = df['temp'].unstack().values

# This is super wonky: convert to epoch --> back to dt object --> remove tz info
times = df['temp'].unstack().index.values
times = mdates.date2num(times)
times = mdates.num2date(times)
times = [t.replace(tzinfo=None) for t in times]

depths = df['temp'].unstack().columns.values
ww_xx,ww_yy = np.meshgrid(times,depths)

# For converting from numpy.datetime64 to datetime object
ns = 1e-9 # number of seconds in a nanosecond

__Loop through each time to make an image stack for an animation__

In [ ]:
single_day = ds.sel(time=slice('20211001','20211101'))

for i in range(single_day.time.shape[0]):
    u = single_day.isel(time=i).water_u.values
    v = single_day.isel(time=i).water_v.values
    magnitude = (u ** 2 + v ** 2) ** 0.5
    time = single_day.isel(time=i).time.values
    time = dt.datetime.utcfromtimestamp(time.astype(int) * ns)

    fig = plt.figure()
    fig.set_size_inches(7,13)
    gs = fig.add_gridspec(5, 2)

    ax = fig.add_subplot(gs[0:3, :], projection=ccrs.PlateCarree())
    ax.set_extent([-122.3, -121.75, 36.4, 37.15], crs=ccrs.PlateCarree())
    ax.coastlines(resolution='auto', color='k')
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    cax = ax.quiver(xx,yy,u,v,magnitude, cmap=cmocean.cm.solar_r, scale=3,width=.0075)

    ## Plot Contour ##
    ax2 = fig.add_subplot(gs[3, :],)
    cax = ax2.contourf(ww_xx,ww_yy, temp_gridded.T)
    ax2.invert_yaxis()
    ax2.text(time,-1,"↓",color='r',size=22,weight='bold')
    plt.colorbar(cax,location='top',pad=.175)

    
    ## 
    ax.set_title("{}".format(time))
    fig.autofmt_xdate()
    ## Save Plots
    
    # plt.savefig(f'/Users/patrick/OneDrive/UCSC/IFCB/figures/hfr-stream-plots/{str(i).zfill(3)}.png',bbox_inches='tight')
    # plt.close()
    # if i % 24 == 0:
        # print(f"{i} out of {single_day.time.shape[0]} completed!")

In [ ]:
single_day = ds.sel(time=slice('20211001','20211101'))
single_day.time.shape

For creating GIFs use the 'convert' software in the CLI

`convert -delay 25 $(ls *.png | sort -V) ../out.gif`